In [1]:
#import tweepy
#import twint
#import nltk
#from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer
#from nltk.stem import Cistem
#from nltk.stem.snowball import GermanStemmer
#from nltk.tokenize import TweetTokenizer
#import re
#import regex as re
#import emoji
#import datetime
#import glob
import gensim as gs
import logging

import io
import os
import random
import string
#import unicodedata
#import time
import sys
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#from wordcloud import WordCloud
#import seaborn as sns
#from pprint import pprint

#import tensorflow as tf
#import tensorflow_datasets as tfds
#from tensorflow.keras import Model
#from tensorflow.keras import Sequential
#from tensorflow.keras.initializers import Constant
#from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
#from tensorflow.keras import layers

#import tqdm

# Workaround for Jupyter's "This event loop is already running" exception
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Check for GPU availability
#if tf.test.gpu_device_name():
#    print(tf.test.gpu_device_name())
#    print("GPU available.")
#else:
#    print("GPU not available.")
    
# Print numpy version
print(np.__version__) # Gensim needs a numpy version of >1.20 to work. Therefore we need two different environments...

1.20.2


In [3]:
#handle="FiatPecunia"
handle="Erdayastronaut"
# Read the tweets dataset
#df = pd.read_csv('{}/preproc/tweets.csv'.format(handle))
# Train the embedding only on the training data set in order to prevent leaks to the validation and test sets
df = pd.read_csv('{}/preproc/train.csv'.format(handle))
#df = df['tweet']
df.head()

,bin,rating,tweet
0,1,0.528947,starship upda liv
1,1,0.593039,forbidd molossia spinach tobacco torpedo playi...
2,1,0.512920,week wild bou
3,1,0.615915,running 4th halfway poi rac cautio lap 43 pitt...
4,0,0.000000,emily chai percy awesom wan lear wai new image...


In [4]:
# Gensim needs an iterator yielding a list of tokens for each tweet.

for index, row in df.iterrows():
    print(row['tweet'].split())
    break

['starship', 'upda', 'liv']


In [5]:
class Corpus:
    """ path must point to a CSV file with columns: rating,tweet,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,video """
    def __init__(self, path):
        #self.df = pd.read_csv(path,
        #             encoding='utf-8',
        #             dtype={'rating': float,
        #                    'tweet': 'string',
        #                    'photos': 'bool',
        #                    'replies_count': int,
        #                    'retweets_count': int,
        #                    'likes_count': int,
        #                    'hashtags': object,
        #                    'cashtags': object,
        #                    'video': int})
        self.df = pd.read_csv(path,
                     encoding='utf-8',
                     dtype={'rating': float,
                            'tweet': 'string',
                            'photos': 'bool',
                            'replies_count': float,
                            'retweets_count': float,
                            'likes_count': float,
                            'hashtags': object,
                            'cashtags': object,
                            'video': int})
        
        #print(self.df.loc[self.df['tweet'].isnull()])
        # Remove empty tweets
        self.df = self.df.dropna(subset=['tweet'])
    
    def __iter__(self):
        for index, row in self.df.iterrows():
            yield row['tweet'].split()

In [7]:
embedding_dim = 256 #32 #256 #128
seed = 42
epochs = 5 #2 #100 #5

#corpus = Corpus('{}/preproc/dataset.csv'.format(handle))
corpus = Corpus('{}/preproc/train.csv'.format(handle))

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = gs.models.Word2Vec(sentences=corpus,
                           min_count=5,
                           sg=1,
                           seed=seed,
                           vector_size=embedding_dim,
                           workers=8,
                           epochs=epochs
                          )

2021-08-14 01:47:38,491 : INFO : collecting all words and their counts
2021-08-14 01:47:38,522 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-08-14 01:47:39,175 : INFO : PROGRESS: at sentence #10000, processed 109717 words, keeping 17102 word types
2021-08-14 01:47:39,818 : INFO : PROGRESS: at sentence #20000, processed 220636 words, keeping 25178 word types
2021-08-14 01:47:40,464 : INFO : PROGRESS: at sentence #30000, processed 331159 words, keeping 31538 word types
2021-08-14 01:47:41,131 : INFO : PROGRESS: at sentence #40000, processed 442251 words, keeping 37042 word types
2021-08-14 01:47:41,816 : INFO : PROGRESS: at sentence #50000, processed 551985 words, keeping 41946 word types
2021-08-14 01:47:42,614 : INFO : PROGRESS: at sentence #60000, processed 661649 words, keeping 46443 word types
2021-08-14 01:47:43,417 : INFO : PROGRESS: at sentence #70000, processed 772103 words, keeping 50577 word types
2021-08-14 01:47:44,078 : INFO : PROGRESS: at 

In [8]:
model.save('{}/preproc/gensim-w2v-model.gs'.format(handle))

2021-08-14 01:52:55,136 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'Erdayastronaut/preproc/gensim-w2v-model.gs', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-08-14T01:52:55.136095', 'gensim': '4.0.1', 'python': '3.9.5 (default, Jun  4 2021, 12:28:51) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.97-gentoo-x86_64-Intel-R-_Core-TM-_i7-4770K_CPU_@_3.50GHz-with-glibc2.33', 'event': 'saving'}
2021-08-14 01:52:55,136 : INFO : not storing attribute cum_table
2021-08-14 01:52:55,217 : INFO : saved Erdayastronaut/preproc/gensim-w2v-model.gs


In [9]:
#model.wv.get_keras_embedding(train_embeddings=False)
model.wv.save('{}/preproc/gensim_w2v_vectors.gs'.format(handle))

2021-08-14 01:52:57,671 : INFO : KeyedVectors lifecycle event {'fname_or_handle': 'Erdayastronaut/preproc/gensim_w2v_vectors.gs', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-08-14T01:52:57.671796', 'gensim': '4.0.1', 'python': '3.9.5 (default, Jun  4 2021, 12:28:51) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.97-gentoo-x86_64-Intel-R-_Core-TM-_i7-4770K_CPU_@_3.50GHz-with-glibc2.33', 'event': 'saving'}
2021-08-14 01:52:57,710 : INFO : saved Erdayastronaut/preproc/gensim_w2v_vectors.gs


In [9]:
# For some weird reason, gensim can't decode its own file...

#model = gs.models.KeyedVectors.load_word2vec_format('preproc/gensim-w2v-model.gs', binary=False)

2021-07-11 18:11:50,250 : INFO : loading projection weights from preproc/gensim-w2v-model.gs


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [10]:
# So we have to save our own dataframe...

#print(model.wv[0])
#print(model.wv.index_to_key[0])
#print(model.wv.key_to_index['mal'])

vocab_size = len(model.wv.key_to_index)
tmp = {}

for i in range(vocab_size):
    word = model.wv.index_to_key[i]
    vec  = model.wv[i]
    
    tmp[word] = vec
    #newDF = newDF.append({word: vec}, ignore_index=True)

newDF = pd.DataFrame.from_dict({'word': [*tmp], 'vec': [*tmp.values()]})
#newDF = newDF.append(tmp, ignore_index=True)
#for i in range(vocab_size):
#    vec = model.wv[i]
#    if(vec is not None):
#        mat[i] = vec

#mat

newDF.head()

newDF.to_pickle('{}/preproc/gensim_w2v_dict.pkl'.format(handle))

In [11]:
for index, word in enumerate(model.wv.index_to_key):
    if index==10:
        break
    print(f"word #{index}/{len(model.wv.index_to_key)} is {word}")

word #0/36333 is spac
word #1/36333 is launch
word #2/36333 is new
word #3/36333 is today
word #4/36333 is amp
word #5/36333 is fir
word #6/36333 is nasa
word #7/36333 is day
word #8/36333 is one
word #9/36333 is missio


In [12]:
try:
    vec_cameroon = model.wv['cameroon']
except KeyError:
        print("'cameroon' does not appear in this model")

'cameroon' does not appear in this model


In [13]:
pairs = [
    ('frau', 'frau'),
    ('frau', 'deutsch'),
    #('frau', 'hat'),
    ('frau', 'mann'),
    ('frau', 'mensch'),
    ('frau', 'eigentlich'),
    ('frau', 'zahnarz'),
    ('mann', 'zahnarz')
    #('mann', 'zahnarz')
]

for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, model.wv.similarity(w1, w2)))

KeyError: "Key 'frau' not present"

In [13]:
print(model.wv.most_similar(positive=['frau', 'deutsch', 'einfach'], topn=10))

[('fuhrungspositio', 0.854916512966156), ('bedach', 0.8507602214813232), ('wozu', 0.8441033959388733), ('istdass', 0.840337872505188), ('diedie', 0.8359581232070923), ('nderfluid', 0.835375964641571), ('ehrenw', 0.8347028493881226), ('tatsachl', 0.8345174789428711), ('diekmann', 0.8343234062194824), ('mihigru', 0.8340935111045837)]


In [17]:
print(model.wv.most_similar(positive=['elo'], topn=10))

[('f9', 0.6931021809577942), ('b1048', 0.6900892853736877), ('koenigsmann', 0.6841893196105957), ('preflow', 0.6839394569396973), ('musk', 0.6830636262893677), ('sixteenth', 0.6705995798110962), ('rtf', 0.6685316562652588), ('mostflow', 0.6650721430778503), ('ofcourseistillloveyou', 0.664594829082489), ('b10517', 0.6589335799217224)]


In [14]:
print(model.wv.most_similar(positive=['frau', 'mann', 'einfach'], topn=10))

[('fuhrungspositio', 0.8700457215309143), ('menstruier', 0.8601241707801819), ('cis', 0.8567571640014648), ('nderfluid', 0.8481584191322327), ('diekmann', 0.8432794213294983), ('transg', 0.8415762186050415), ('diskriminier', 0.8389608263969421), ('heterosexuell', 0.8335394859313965), ('hetero', 0.8319412469863892), ('transsexuell', 0.8256118297576904)]


In [15]:
print(model.wv.most_similar(positive=['deutschla', 'deutsch'], topn=10))

[('staatsangehorigkei', 0.8638187646865845), ('dt', 0.8447890281677246), ('alliier', 0.8401046991348267), ('amtssprach', 0.8331677317619324), ('islamisier', 0.8321091532707214), ('staatsburg', 0.8303978443145752), ('schutzzo', 0.8299842476844788), ('chil', 0.8296916484832764), ('masseneinwanderung', 0.8284094929695129), ('wwf', 0.82839435338974)]


In [16]:
print(model.wv.most_similar(positive=['frau'], topn=10))

[('mann', 0.8960163593292236), ('weiblich', 0.7824152112007141), ('schwul', 0.7780516743659973), ('fuhrungspositio', 0.7742252945899963), ('menstruier', 0.7705603241920471), ('fr', 0.768467128276825), ('lesbisch', 0.7672699093818665), ('cis', 0.763115644454956), ('transsexuell', 0.7594537138938904), ('transg', 0.7569398880004883)]


In [17]:
print(model.wv.most_similar(positive=['zahnarz'], topn=10))

[('hausarzti', 0.9121438264846802), ('ungel', 0.9001254439353943), ('krankenpfleg', 0.8994037508964539), ('hebamm', 0.8988788723945618), ('kassiererinn', 0.8953143358230591), ('diplom', 0.8911609053611755), ('artig', 0.8911332488059998), ('steuerbera', 0.8908710479736328), ('anzuruf', 0.8900642991065979), ('apothekerinn', 0.8887056112289429)]


In [18]:
print(model.wv.most_similar(positive=['end'], topn=10))

[('word', 0.7806714773178101), ('wiederkomm', 0.7724742293357849), ('beginn', 0.7633434534072876), ('bang', 0.7630654573440552), ('ruckfall', 0.7624064087867737), ('ligh', 0.752379298210144), ('ausruf', 0.7379752397537231), ('fin', 0.735625684261322), ('wiederkehr', 0.7333568930625916), ('uberwu', 0.7310155630111694)]


In [19]:
print(model.wv.most_similar(positive=['afd'], topn=10))

[('npd', 0.8942745327949524), ('altpartei', 0.8857232332229614), ('ramelow', 0.8720541596412659), ('cdu', 0.8675476312637329), ('bt', 0.8491915464401245), ('faschi', 0.8455417156219482), ('kalbitz', 0.8439045548439026), ('koalier', 0.843462347984314), ('flugel', 0.8431350588798523), ('kemmerich', 0.8429465889930725)]


In [20]:
print(model.wv.most_similar(positive=['cdu'], topn=10))

[('fdp', 0.9483650922775269), ('spd', 0.9469727277755737), ('koalitio', 0.9168410301208496), ('cducsu', 0.9045897126197815), ('csu', 0.8953536152839661), ('volkspartei', 0.8888271450996399), ('kanzlerkandida', 0.8878781199455261), ('linkspartei', 0.8877407908439636), ('unio', 0.8864145278930664), ('zweitstimm', 0.8800886869430542)]


In [21]:
print(model.wv.most_similar(positive=['csu'], topn=10))

[('cducsu', 0.9011437296867371), ('cdu', 0.8953537344932556), ('koalitionspar', 0.892500102519989), ('unio', 0.8887526392936707), ('zweitstimm', 0.8885459899902344), ('klingbeil', 0.8881409764289856), ('linksruck', 0.8836401104927063), ('spdvorsitz', 0.8836240172386169), ('koalitio', 0.8826844096183777), ('grunro', 0.8825855851173401)]


In [22]:
print(model.wv.most_similar(positive=['fdp'], topn=10))

[('cdu', 0.9483649730682373), ('spd', 0.9283211827278137), ('cducsu', 0.9180498719215393), ('zweitstimm', 0.8811129331588745), ('fw', 0.8774774074554443), ('koalitio', 0.8748432993888855), ('unio', 0.8732982873916626), ('erststimm', 0.8687703013420105), ('csu', 0.8670068979263306), ('partei', 0.8645967841148376)]


In [23]:
print(model.wv.most_similar(positive=['lkr'], topn=10))

[('spdfraktio', 0.9688435792922974), ('thuringisch', 0.9660668969154358), ('23mehrhei', 0.9618275165557861), ('273', 0.9613721370697021), ('blockadehaltung', 0.9607844352722168), ('klimali', 0.9599866271018982), ('obkandida', 0.958749532699585), ('stimmenanteil', 0.9574214816093445), ('535', 0.9569948315620422), ('545', 0.9547545909881592)]


In [24]:
print(model.wv.most_similar(positive=['gru'], topn=10))

[('vielflieg', 0.8438335657119751), ('spd', 0.8301751613616943), ('grr', 0.8293755054473877), ('cducsu', 0.8144592046737671), ('grunro', 0.812215268611908), ('90die', 0.8096510767936707), ('cdu', 0.8045805096626282), ('grunrotro', 0.7999962568283081), ('sozialdemokratisch', 0.7984183430671692), ('koalier', 0.7974988222122192)]


In [25]:
print(model.wv.most_similar(positive=['spd'], topn=10))

[('cdu', 0.946972668170929), ('fdp', 0.9283212423324585), ('cducsu', 0.906523585319519), ('koalitio', 0.8867268562316895), ('csu', 0.878409743309021), ('unio', 0.8595377802848816), ('zweitstimm', 0.8590611815452576), ('cdul', 0.8576090335845947), ('linkspartei', 0.8564978837966919), ('rotgru', 0.8564102649688721)]


In [26]:
print(model.wv.most_similar(positive=['dielink'], topn=10))

[('fuhrungsrieg', 0.975919783115387), ('ortsvorsteh', 0.9727416038513184), ('landescdu', 0.9717168211936951), ('abweg', 0.9713981747627258), ('dehm', 0.9702876806259155), ('wendehal', 0.9695779085159302), ('mitwahl', 0.969078779220581), ('parteiaustritt', 0.9680103659629822), ('afdstimm', 0.9674060344696045), ('vizechef', 0.967309296131134)]


In [27]:
print(model.wv.most_similar(positive=['ezb'], topn=10))

[('lder', 0.927870512008667), ('zentralbank', 0.8833544254302979), ('kredi', 0.8832144737243652), ('negativzi', 0.8806372880935669), ('staatshaushal', 0.8802966475486755), ('subventionier', 0.8802087903022766), ('entwicklungshilf', 0.8775474429130554), ('anleih', 0.8748691082000732), ('zuruckzahl', 0.8735208511352539), ('genleistung', 0.8732107877731323)]


In [28]:
print(model.wv.most_similar(positive=['ezb', 'lagard', 'euro'], topn=10))

[('boni', 0.9215742349624634), ('rucklag', 0.9185839891433716), ('rettungspak', 0.9184892177581787), ('euhaushal', 0.916375458240509), ('bundeshaushal', 0.915538489818573), ('950', 0.9139079451560974), ('burgschaf', 0.9132876396179199), ('milliardenhoh', 0.9102363586425781), ('finanzmini', 0.9089623689651489), ('hilfspak', 0.9056674242019653)]


In [29]:
print(model.wv.most_similar(positive=['shitcoi', 'euro'], topn=10))

[('uberwie', 0.9282001256942749), ('jahreseinkomm', 0.9274791479110718), ('lappisch', 0.9273765087127686), ('ergau', 0.9206568002700806), ('zehnfach', 0.9205203652381897), ('950', 0.9204089045524597), ('dienstwag', 0.9194821715354919), ('einmalzahlung', 0.9194193482398987), ('coronasoforthilf', 0.9189321398735046), ('honorar', 0.9173020124435425)]


In [30]:
print(model.wv.most_similar(positive=['gold'], topn=10))

[('silb', 0.9022963047027588), ('asche', 0.8018727898597717), ('leih', 0.796103298664093), ('eich', 0.7865476608276367), ('zitro', 0.778553307056427), ('munz', 0.7716904878616333), ('kalorie', 0.7714682221412659), ('schipp', 0.7641925811767578), ('ferrari', 0.762031614780426), ('pump', 0.761747419834137)]


In [31]:
print(model.wv.doesnt_match(['ezb', 'rechtigkei', 'zukunf', 'stabilita']))

ezb


In [32]:
print(model.wv.doesnt_match(['frau', 'mann', 'land', 'einfach', 'lass', 'kind']))

einfach


In [33]:
print(model.wv.doesnt_match(['mal', 'einfach', 'land', 'deutschla', 'lass', 'kind']))

mal
